### testing preparation methods done by LASTRO

In [19]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import glob
import os
from astropy.io import fits
import importlib.util
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import matplotlib.backends.backend_pdf
import re

In [20]:
def prepare(images):
    images[images == 100] = 0.0
    mean = np.mean(images)
    std = np.std(images)
    images = (images - mean) / std
    return images, mean, std

In [21]:
# original 4 band images to check method
files4 = sorted(glob.glob('/Users/mac/Desktop/LBNL/challenge_data/GroundBasedTraining/npz/*.npz'))
x4 = np.zeros((len(files4), 101, 101, 4))
for i,f in enumerate(files4):
    with np.load(f) as data:
        im = data['image']
    x4[i] = im
print(x4.shape)

(20000, 101, 101, 4)


In [22]:
# new 3 band images to get mean and stf
files3 = sorted(glob.glob('/Users/mac/Desktop/LBNL/lensfinder-euclid-master/npz_3band_groundbased/*.npz'))
x3 = np.zeros((len(files3), 101, 101, 3))
for i,f in enumerate(files3):
    with np.load(f) as data:
        im = data['image']
    x3[i] = im
print(x3.shape)

(20000, 101, 101, 3)


In [23]:
x4mean = np.mean(x4)
x3mean = np.mean(x3)
x4std = np.std(x4)
x3std = np.std(x3)
x4p, x4pmean, x4pstd = prepare(x4)
x3p, x3pmean, x3pstd = prepare(x3)

In [24]:
print('- UNPREPARED -')
print('\n4 band\nmean = {}\nstd = {}'.format(x4mean, x4std))
print('\n3 band\nmean = {}\nstd = {}'.format(x3mean, x3std))

print('\n- PREPARED -')
print('\n4 band\nmean = {} (after mask) {} (after normed)\nstd = {} (after mask) {} (after normed)'.format(x4pmean, np.mean(x4p), x4pstd, np.std(x4p)))
print('\n3 band\nmean = {} (after mask) {} (after normed)\nstd = {} (after mask) {} (after normed)'.format(x3pmean, np.mean(x3p), x3pstd, np.std(x3p)))

- UNPREPARED -

4 band
mean = 1.1923566317041112
std = 10.854213413845873

3 band
mean = 1.1340948599827658
std = 10.58882972035045

- PREPARED -

4 band
mean = 1.6649487250414204e-12 (after mask) -1.7679912800684978e-16 (after normed)
std = 5.090208456258302e-11 (after mask) 0.999999999999995 (after normed)

3 band
mean = 2.020098959059707e-12 (after mask) 5.295698483093407e-16 (after normed)
std = 5.3907523393359274e-11 (after mask) 1.0000000000000016 (after normed)


In [16]:
# checking stuff for data being used to train 3 band LASTRO model

f_train = files3[1500:5000]
f_test = files3[:1500]

train_lens = 0
train_nonlens = 0
test_lens = 0
test_nonlens = 0

for f in f_train:
    with np.load(f) as data:
        if data['is_lens'] == 1:
            train_lens += 1
        if data['is_lens'] == 0:
            train_nonlens += 1
for f in f_test:
    with np.load(f) as data:
        if data['is_lens'] == 1:
            test_lens += 1
        if data['is_lens'] == 0:
            test_nonlens += 1

In [18]:
print('train_lens = {}/3500\ntrain_nonlens = {}/3500\ntest_lens = {}/1500\ntest_nonlens = {}/1500'.format(train_lens,train_nonlens,test_lens,test_nonlens))

train_lens = 1390/3500
train_nonlens = 2110/3500
test_lens = 629/1500
test_nonlens = 871/1500


In [33]:
for i in range(0, 72, 12):
    print('{}/{}'.format(i, 72))

0/72
12/72
24/72
36/72
48/72
60/72
